In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import tensorflow.compat.v1 as tf
import skimage
import skimage.io

2023-10-29 12:12:42.407122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


: 

In [ ]:
# def _image_feature(value):
#     """Returns a bytes_list from a string / byte."""
#     return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

def _image_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.tobytes()]))

def chunk_list(lst: list, chunk_size: int):
    assert type(chunk_size) == int
    assert type(lst) == list
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]  

def single_tf_records(filenames, output_dir, tf_record_name):
    images = []
    for filename in filenames:
        images.append(skimage.io.imread(filename)) # read image from filename
    images_array = np.array(images)

    TFRecord_filename = os.path.join(output_dir, str(tf_record_name) + '.tfrecords')
    print('Writing', TFRecord_filename)
    writer = tf.python_io.TFRecordWriter(TFRecord_filename)
    for image in images_array:
        try: 
            feature = tf.train.Features(feature={'image': _image_feature(image)})
            example = tf.train.Example(features=feature)
            writer.write(example.SerializeToString())
        except: 
            print("lol")
            




def convert_n_save_jpgs_to_TFRecords(input_dir, output_dir, chunk_size):
    try: os.mkdir(output_dir)
    except: pass
    filenames = tf.io.gfile.glob(input_dir)
    filenames_chunk_list = chunk_list(filenames, chunk_size)
    for index, filenames_chunk in enumerate(filenames_chunk_list):
#         if index > 11:
        if index <= 11:
            pass
        else:
            single_tf_records(filenames_chunk, output_dir, index)
        

In [ ]:
PWS_FACES_PATH = "../../data/PWS-IMAGES*.jpg"
convert_n_save_jpgs_to_TFRecords(PWS_FACES_PATH, "pws-images-tfrecord", 47)

In [ ]:
HUMAN_FACES_PATH = "/kaggle/input/human-faces/Humans/*.jpg"
convert_n_save_jpgs_to_TFRecords(HUMAN_FACES_PATH, "human-faces-images-tfrecord", 350)